In [1]:
%reload_ext autoreload
%autoreload 2

from IPython.display import clear_output

In [2]:
import os
import sys

sys.path.append('..')

from src.data.boardgame_resource import BoardgameResource

In [ ]:
from bs4 import BeautifulSoup
import re
import requests

sitemaps = requests.get('http://www.boardgamegeek.com/sitemapindex').content
sitemaps = BeautifulSoup(sitemaps, 'lxml').select('sitemapindex > sitemap > loc')
sitemaps = [str(s.get_text()) for s in sitemaps]
sitemaps = [s.replace('\n', '') for s in sitemaps]
sitemaps = [s.replace('\t', '') for s in sitemaps]
sitemaps = [s for s in sitemaps if 'geekitems_boardgame_' in s]

In [ ]:
boardgame_urls = []
for sitemap in sitemaps:
    sitemap = requests.get(sitemap).content
    urls = BeautifulSoup(sitemap, 'lxml').select('urlset > url > loc')
    urls = [str(u.get_text()) for u in urls]
    urls = [u.replace('\n', '') for u in urls]
    urls = [u.replace('\t', '') for u in urls]
    boardgame_urls.extend(urls)

In [ ]:
boardgame_ids = [int(re.compile('[0-9]+').search(u).group(0)) for u in boardgame_urls]

In [ ]:
import pickle

pickle.dump(boardgame_ids, open('bgids.p', 'w+'))

In [3]:
import pickle

boardgame_ids = pickle.load(open('bgids.p', 'rb'))
len(boardgame_ids)

86855

In [4]:
from src.data.crawler import Crawler
from src.data.schema import Schema
save_dir = os.path.join(os.path.dirname(os.getcwd()), 'data', 'raw')

In [178]:
raw_schema = \
{
    'stats': 'https://boardgamegeek.com/xmlapi2/thing?id=%(id)s&stats=1'
}

for i, id in enumerate(boardgame_ids):
    
    clear_output()
    print i, id
    
    schema = Schema(raw_schema, {'id': id}, save_dir)
    Crawler(schema).crawl('stats')

0 1


NameError: name 'Crawler' is not defined

In [5]:
from src.data.parser import Parser
from src.data.schema import Schema
save_dir = os.path.join(os.path.dirname(os.getcwd()), 'data', 'raw')

import pandas as pd

In [6]:
raw_schema = \
{
    'stats': 'https://boardgamegeek.com/xmlapi2/thing?id=%(id)s&stats=1'
}

schema = Schema(raw_schema, {'id': 17}, save_dir)
parser = Parser(schema)

In [10]:
df = pd.DataFrame(columns=list(parser.flatten()))

In [12]:
raw_schema = \
{
    'stats': 'https://boardgamegeek.com/xmlapi2/thing?id=%(id)s&stats=1'
}

for i, id in enumerate(boardgame_ids):
    
    if i % 1000 == 0:
        clear_output()
        print i, id
    
    schema = Schema(raw_schema, {'id': id}, save_dir)
    row = Parser(schema).flatten()
    df = df.append(row, ignore_index=True)

86000 208006


In [13]:
from datetime import datetime
datetime.now()

datetime.datetime(2016, 11, 6, 10, 25, 20, 949610)

In [14]:
import dill as pickle

with open('gambit_raw_abt_20161106.p', 'wb+') as f:
    pickle.dump(df, f)

In [25]:
mechanics = set().union(*df['mechanics'])

In [26]:
mechanics

{'Acting',
 'Action / Movement Programming',
 'Action Point Allowance System',
 'Area Control / Area Influence',
 'Area Enclosure',
 'Area Movement',
 'Area-Impulse',
 'Auction/Bidding',
 'Betting/Wagering',
 'Campaign / Battle Card Driven',
 'Card Drafting',
 'Chit-Pull System',
 'Co-operative Play',
 'Commodity Speculation',
 'Crayon Rail System',
 'Deck / Pool Building',
 'Dice Rolling',
 'Grid Movement',
 'Hand Management',
 'Hex-and-Counter',
 'Line Drawing',
 'Memory',
 'Modular Board',
 'Paper-and-Pencil',
 'Partnerships',
 'Pattern Building',
 'Pattern Recognition',
 'Pick-up and Deliver',
 'Player Elimination',
 'Point to Point Movement',
 'Press Your Luck',
 'Rock-Paper-Scissors',
 'Role Playing',
 'Roll / Spin and Move',
 'Route/Network Building',
 'Secret Unit Deployment',
 'Set Collection',
 'Simulation',
 'Simultaneous Action Selection',
 'Singing',
 'Stock Holding',
 'Storytelling',
 'Take That',
 'Tile Placement',
 'Time Track',
 'Trading',
 'Trick-taking',
 'Variable P